## 1. 只考虑两种题目中给定的特征

In [2]:
import pandas as pd
import statsmodels.api as sm

# 1. 读取处理后的数据
df_raw = pd.read_csv('data_processed.csv')
df = pd.read_csv('data_processed_std.csv')


# 2. 设置因变量（目标变量）
target_col = 'Y染色体浓度' 

# 3. 选择数值型自变量（排除目标列）
numeric_cols = ['检测孕周_周数','孕妇BMI']

# 4. 定义 X 和 y
X = df[numeric_cols]
y = df[target_col]

# 5. 添加常数项（截距）
X = sm.add_constant(X)

# 6. 建立 OLS 回归模型
model = sm.OLS(y, X).fit()

# 7. 输出回归结果
print(model.summary())


# 8. 提取显著性变量（p < 0.05）
p_values = model.pvalues
significant_vars = p_values[p_values < 0.05].index.tolist()

# 去掉常数项
if 'const' in significant_vars:
    significant_vars.remove('const')

print("\n显著性变量（p < 0.05）：")
print(significant_vars)

# 9. 保存回归结果到文件
summary_df = pd.DataFrame({
    '变量': model.params.index,
    '系数': model.params.values,
    'p值': model.pvalues.values,
    '显著性': ['显著' if p < 0.05 else '不显著' for p in model.pvalues.values]
})
summary_df.to_csv('regression_results_only2.csv', index=False, encoding='utf-8-sig')

print("\n回归结果已保存到 regression_results_only2.csv")


                            OLS Regression Results                            
Dep. Variable:                 Y染色体浓度   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     18.45
Date:                Fri, 05 Sep 2025   Prob (F-statistic):           1.42e-08
Time:                        20:30:48   Log-Likelihood:                 1784.0
No. Observations:                 877   AIC:                            -3562.
Df Residuals:                     874   BIC:                            -3548.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0797      0.001     74.159      0.0

In [5]:
# 获取 95% 置信区间
conf_int_95 = model.conf_int(alpha=0.05)  # alpha=0.05 表示 95% 置信水平
conf_int_95.columns = ['下限', '上限']
print(conf_int_95)

# 判断置信区间是否包含 0
significant_ci_vars = conf_int_95[
    (conf_int_95['下限'] > 0) | (conf_int_95['上限'] < 0)
].index.tolist()
# 去掉常数项
if 'const' in significant_ci_vars:
    significant_ci_vars.remove('const')
print("\n置信区间法显著变量（95% CI 不包含 0）：")
print(significant_ci_vars)

               下限        上限
const    0.076971  0.087352
检测孕周_周数  0.001750  0.006394
年龄      -0.005096 -0.000624
孕妇BMI   -0.009589 -0.004569
IVF妊娠   -0.019281  0.003154
怀孕次数    -0.004950  0.001328
生产次数    -0.002047  0.006229

置信区间法显著变量（95% CI 不包含 0）：
['检测孕周_周数', '年龄', '孕妇BMI']


## 2. 考虑多个可能的特征

In [1]:
import pandas as pd
import statsmodels.api as sm

# 1. 读取处理后的数据
df_raw = pd.read_csv('data_processed.csv')
df = pd.read_csv('data_processed_std.csv')


# 2. 设置因变量（目标变量）
target_col = 'Y染色体浓度'  # 你可以改成其他目标变量

# 3. 选择数值型自变量（排除目标列）
numeric_cols = ['检测孕周_周数','年龄','孕妇BMI','IVF妊娠','怀孕次数','生产次数']

# 4. 定义 X 和 y
X = df[numeric_cols]
y = df[target_col]

# 5. 添加常数项（截距）
X = sm.add_constant(X)

# 6. 建立 OLS 回归模型
model = sm.OLS(y, X).fit()

# 7. 输出回归结果
print(model.summary())


# 输出model的MAE等指标，由于是OLS模型，所以不适合计算MAE、MSE、RMSE


# 8. 提取显著性变量（p < 0.05）
p_values = model.pvalues
significant_vars = p_values[p_values < 0.05].index.tolist()

# 去掉常数项
if 'const' in significant_vars:
    significant_vars.remove('const')

print("\n显著性变量（p < 0.05）：")
print(significant_vars)

# 9. 保存回归结果到文件
summary_df = pd.DataFrame({
    '变量': model.params.index,
    '系数': model.params.values,
    'p值': model.pvalues.values,
    '显著性': ['显著' if p < 0.05 else '不显著' for p in model.pvalues.values]
})
summary_df.to_csv('regression_results.csv', index=False, encoding='utf-8-sig')

print("\n回归结果已保存到 regression_results.csv")


                            OLS Regression Results                            
Dep. Variable:                 Y染色体浓度   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     8.214
Date:                Fri, 05 Sep 2025   Prob (F-statistic):           1.15e-08
Time:                        20:52:29   Log-Likelihood:                 1790.0
No. Observations:                 877   AIC:                            -3566.
Df Residuals:                     870   BIC:                            -3533.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0822      0.003     31.068      0.0

## 第二次分析:

这种现象在实际数据分析中很常见，原因可能有：

多重共线性（Multicollinearity）

BMI 和其他变量（如身高、体重）高度相关
在 OLS 中，如果自变量之间高度相关，系数估计会不稳定，p 值会变大
可能 BMI 的信息已经被“身高+体重”解释掉了
模型设定不当

OLS 假设是线性关系，但 BMI 和因变量可能是非线性关系
如果真实关系是曲线型，线性模型会低估它的显著性
噪声和样本量

样本量不足或噪声较大，也会导致 p 值偏大
多重比较问题

你有很多变量（>20 个），在多次显著性检验中，必然会有一些变量 p 值不显著，即使它们在实际中有影响


### 方差膨胀因子（VIF）:

去除部分

In [4]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1. 读取处理后的数据
df = pd.read_csv('data_processed_std.csv')

X = df[['检测孕周_周数','年龄','孕妇BMI','IVF妊娠','怀孕次数','生产次数']]  # 自变量
X = sm.add_constant(X)
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data.sort_values("VIF", ascending=False))

   feature       VIF
0    const  6.159141
5     怀孕次数  1.628078
6     生产次数  1.608285
2       年龄  1.054616
4    IVF妊娠  1.034690
1  检测孕周_周数  1.017961
3    孕妇BMI  1.017189


### LassoCV

### 用 随机森林 计算特征重要性

In [5]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
X = X.drop(['const'], axis=1)
rf.fit(X, y)
importance = pd.Series(rf.feature_importances_, index=X.columns)
print(importance.sort_values(ascending=False))

孕妇BMI      0.425436
检测孕周_周数    0.268134
年龄         0.188636
怀孕次数       0.063470
生产次数       0.050661
IVF妊娠      0.003665
dtype: float64
